The following cells allow the user to specify the risk-free rate and the market risk premium. 

In [1]:
risk_free_rate = 0.04

In [2]:
daily_risk_free_rate = (1+risk_free_rate)**(1/365)-1

In [3]:
equity_risk_premium = 0.046

In [4]:
daily_equity_risk_premium = (1+equity_risk_premium)**(1/365)-1

In [5]:
yearly_market_return = risk_free_rate + equity_risk_premium

In [6]:
daily_market_return = (1 + yearly_market_return) **(1/365) - 1

In [7]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

SPX was used as the baseline in order to calculate the market varience. 

I used the associated R script to clean the data, calculate the daily log returns, and I read in these as CSV's below. This could have been done in Python, as shown with SPX below. 

EG7 is traded in Stockholm, so the OMXSBPI index was used as a baseline for this.

EPSIL is traded in Athens, so the Athens Composite index was used as a baseline for this.

NLOP only became publicly traded late last year, so SPX returns were modified in keeping to those dates.

In [8]:
SPX_read = pd.read_csv('/Users/diarmuidosullivan/Downloads/^SPX.csv')

current_portfolio = pd.read_csv("/Users/diarmuidosullivan/Downloads/OpenPosition_3_27_2024.csv")[0:12]

log_returns_ex_EG7_NLOP = pd.read_csv("/Users/diarmuidosullivan/Downloads/gen_log_retuns.csv")
log_returns_EG7 = pd.read_csv("/Users/diarmuidosullivan/Downloads/EG7_log_retuns_compare.csv")
log_returns_NLOP = pd.read_csv("/Users/diarmuidosullivan/Downloads/NLOP_log_returns.csv")
log_returns_EPSIL = pd.read_csv("/Users/diarmuidosullivan/Downloads/EPSIL_log_returns.csv")

In [9]:
SPX_daily_log_returns = []
SPX_close = SPX_read['Adj Close']
for i in range (1, len(SPX_read)-1):
    log_returns = math.log(SPX_close[i+1]/SPX_close[i])
    SPX_daily_log_returns.append(log_returns)
SPX_mean_return = np.average(SPX_daily_log_returns)
SPX_varience_return = np.var(SPX_daily_log_returns)

In [10]:
names = current_portfolio['Symbol']
names_new = names.tolist()

This calculates the how much we invested into each stock as a proportion of the total amount we invested in stocks

In [11]:
final_proportions_list = []
Total_Equity_Index = sum(current_portfolio['MarketValue'])
for i in range(len(current_portfolio['MarketValue'])):
    temp = current_portfolio['MarketValue'][i]/Total_Equity_Index
    final_proportions_list.append(temp)

Simply renaming BRK-B to BRK

In [12]:
final_proportions = pd.DataFrame (final_proportions_list, names_new)
final_proportions_1 = final_proportions[0]
final_proportions_dict = final_proportions_1.to_dict()
final_proportions_dict['BRK'] = final_proportions_dict["BRK-B"]
del final_proportions_dict["BRK-B"]

To calculate the expected returns and varience of returns for each asset, I used the single index model. 

I ran a regression analysis on the returns of each asset vs the returns of its associated baseline. 

The equity risk premium (alpha) of each stock, is the intercept of the regression line. This is the log return for each asset on a day where its associated baseline does not change value. 

The beta of each stock is the slope slope of the regression line. This is the change in the value of the asset on a day where there is a one unit change in the value of the associated baseline. 

In order to calculate the varience of return on each asset, I needed to calculated each assets residual standard error of each stocks log returns. This is difficult to do in Python, as neither sklearn nor scikit-learn have in built functions to calculate this, so I did this part in R. x

In [13]:
dictionary_of_alphas = {}
dictionary_of_betas= {}
dictionary_of_standard_deviation_of_error = {'EG7_standard_deviation_of_error' : 0.03284, 
                                             'GFL_standard_deviation_of_error' : 0.01551, 
                                             'BRK_standard_deviation_of_error' : 0.006217,
                                             'DAKT_standard_deviation_of_error' : 0.03023,
                                             'FCFS_standard_deviation_of_error' : 0.0137,
                                             'IDT_standard_deviation_of_error' : 0.01992, 
                                             'IWGFF_standard_deviation_of_error' : 0.02919, 
                                             'XPEL_standard_deviation_of_error' : 0.0271,
                                             'SENEA_standard_deviation_of_error' : 0.02478, 
                                             'CROX_standard_deviation_of_error' : 0.03059, 
                                             'NLOP_standard_deviation_of_error' : 0.03998, 
                                             'EPSIL_standard_deviation_of_error' : 0.02205}

In [14]:
def single_index_model(x):
    alpha_and_beta = LinearRegression()
    alpha_and_beta.fit(log_returns_ex_EG7_NLOP['SPX_Log_Returns'].values.reshape((-1, 1)), log_returns_ex_EG7_NLOP[f'{x}_Log_Returns'])
    
    alpha = alpha_and_beta.intercept_
    beta = alpha_and_beta.coef_
    
    for names_new in x:
        dictionary_of_alphas[f'{x}_alpha'] = alpha_and_beta.intercept_
        dictionary_of_betas[f'{x}_beta'] = alpha_and_beta.coef_
        
    
    
    return alpha, beta


In [15]:
def single_index_model_EG7(x):
    alpha_and_beta = LinearRegression()
    alpha_and_beta.fit(log_returns_EG7['OMXSBPI_Log_Returns'].values.reshape((-1, 1)), log_returns_EG7[f'{x}_Log_Returns'])
    
    alpha = alpha_and_beta.intercept_
    beta = alpha_and_beta.coef_
    
    for names_new in x:
        dictionary_of_alphas[f'{x}_alpha'] = alpha_and_beta.intercept_
        dictionary_of_betas[f'{x}_beta'] = alpha_and_beta.coef_
        
    
    
    return alpha, beta

In [16]:
def single_index_model_NLOP(x):
    alpha_and_beta = LinearRegression()
    alpha_and_beta.fit(log_returns_NLOP['SPX_Log_Returns'].values.reshape((-1, 1)), log_returns_NLOP[f'{x}_Log_Returns'])
    
    alpha = alpha_and_beta.intercept_
    beta = alpha_and_beta.coef_
    
    for names_new in x:
        dictionary_of_alphas[f'{x}_alpha'] = alpha_and_beta.intercept_
        dictionary_of_betas[f'{x}_beta'] = alpha_and_beta.coef_
        
    
    
    return alpha, beta

In [17]:
def single_index_model_EPSIL(x):
    alpha_and_beta = LinearRegression()
    alpha_and_beta.fit(log_returns_EPSIL['ATH_Log_Returns'].values.reshape((-1, 1)), log_returns_EPSIL[f'{x}_Log_Returns'])
    
    alpha = alpha_and_beta.intercept_
    beta = alpha_and_beta.coef_
    
    for names_new in x:
        dictionary_of_alphas[f'{x}_alpha'] = alpha_and_beta.intercept_
        dictionary_of_betas[f'{x}_beta'] = alpha_and_beta.coef_
        
    
    
    return alpha, beta

In [18]:
single_index_model('BRK')
single_index_model('CROX')
single_index_model('DAKT')
single_index_model_EG7('EG7')
single_index_model_EPSIL('EPSIL')
single_index_model('FCFS')
single_index_model('GFL')
single_index_model('IDT')
single_index_model('IWGFF')
single_index_model_NLOP('NLOP')
single_index_model('SENEA')
single_index_model('XPEL')

(-0.001876838220562371, array([1.05101711]))

In [19]:
beta_values = [value for value in dictionary_of_betas.values()]

In [20]:
n = len(beta_values)
covariance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i == j:
                covariance_matrix[i, j] = beta_values[i] ** 2 * SPX_varience_return
        else:
                covariance_matrix[i, j] = beta_values[i] * beta_values[j] * SPX_varience_return



covariance_matrix

array([[ 1.96632176e-05,  5.00090756e-05,  3.51253611e-05,
         6.34765793e-06, -1.57166412e-06,  2.03597364e-05,
         2.82464432e-05,  9.35501291e-06,  2.12126537e-05,
         4.15546613e-05,  1.25831936e-05,  3.40042968e-05],
       [ 5.00090756e-05,  1.27187101e-04,  8.93336419e-05,
         1.61438739e-05, -3.99718254e-06,  5.17805182e-05,
         7.18386251e-05,  2.37924209e-05,  5.39497261e-05,
         1.05685155e-04,  3.20025894e-05,  8.64824611e-05],
       [ 3.51253611e-05,  8.93336419e-05,  6.27461394e-05,
         1.13391298e-05, -2.80753999e-06,  3.63695865e-05,
         5.04579942e-05,  1.67113142e-05,  3.78931941e-05,
         7.42311108e-05,  2.24779701e-05,  6.07435278e-05],
       [ 6.34765793e-06,  1.61438739e-05,  1.13391298e-05,
         2.04914384e-06, -5.07362856e-07,  6.57250735e-06,
         9.11848525e-06,  3.01997482e-06,  6.84784516e-06,
         1.34146293e-05,  4.06209249e-06,  1.09772291e-05],
       [-1.57166412e-06, -3.99718254e-06, -2.8075399

In [21]:
def expected_returns(x):
    return dictionary_of_alphas.get(f'{x}_alpha') + dictionary_of_betas.get(f'{x}_beta') * daily_market_return

In [22]:
print(expected_returns('BRK'))
print(expected_returns('CROX'))
print(expected_returns('DAKT'))
print(expected_returns('EG7'))
print(expected_returns('EPSIL'))
print(expected_returns('FCFS'))
print(expected_returns('GFL'))
print(expected_returns('IDT'))
print(expected_returns('IWGFF'))
print(expected_returns('NLOP'))
print(expected_returns('SENEA'))
print(expected_returns('XPEL'))

[0.00069767]
[-0.00058733]
[0.00140584]
[-0.00263685]
[0.00089096]
[0.00062168]
[-0.00048467]
[0.00030076]
[0.00066124]
[0.00460101]
[-0.00012453]
[-0.00163925]


In [23]:
def variance_of_return(x):
    return (dictionary_of_betas.get(f'{x}_beta')**2)*SPX_varience_return + dictionary_of_standard_deviation_of_error.get(f'{x}_standard_deviation_of_error')

In [24]:
portfolio_varience = np.dot(np.array(final_proportions_list).T, np.dot(covariance_matrix, final_proportions_list)) 
portfolio_varience

2.8065828934638073e-05

In [25]:
annual_sd = np.sqrt(portfolio_varience*252)
annual_sd

0.08409868543282228

In [26]:
portfolio_expected_return= expected_returns('BRK')*final_proportions_dict.get('BRK')+expected_returns('CROX')*final_proportions_dict.get('CROX')+expected_returns('DAKT')*final_proportions_dict.get('DAKT')+expected_returns('EG7')*final_proportions_dict.get('EG7')+expected_returns('EPSIL')*final_proportions_dict.get('EPSIL')+expected_returns('FCFS')*final_proportions_dict.get('FCFS')+expected_returns('GFL')*final_proportions_dict.get('GFL')+expected_returns('IDT')*final_proportions_dict.get('IDT')+expected_returns('IWGFF')*final_proportions_dict.get('IWGFF')+expected_returns('NLOP')*final_proportions_dict.get('NLOP')+expected_returns('SENEA')*final_proportions_dict.get('SENEA')+expected_returns('XPEL')*final_proportions_dict.get('XPEL')

In [27]:
portfolio_expected_return

array([0.00055018])

In [28]:
(portfolio_expected_return+1)**252

array([1.14867421])

95% VaR

There is a 95% chance that the returns of our portfolio over the next year will be above 1.033%

In [29]:
(((portfolio_expected_return+1)**252) - 1.645*annual_sd)

array([1.01033187])